# 04 · Modelado Predictivo

Objetivo: entrenar modelos de retención (clasificación) y frecuencia (regresión) con métricas clave.

In [ ]:
import os, pandas as pd
from pathlib import Path
import sys

# Resolver raíz del proyecto de forma robusta

def find_root(start=None):
    p = Path(start or Path.cwd()).resolve()
    for _ in range(6):
        if (p / 'requirements.txt').exists() and (p / 'src').exists():
            return p
        if (p / '.git').exists() and (p / 'src').exists():
            return p
        p = p.parent
    return Path.cwd()

ROOT = find_root()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.modeling import train_retention_model, train_frequency_model, save_model

FEAT_PATH = ROOT / 'data' / 'processed' / 'model_features.csv'

print('cwd:', os.getcwd())
print('ROOT:', ROOT)
print('Features existe?', FEAT_PATH.exists(), '->', FEAT_PATH)

try:
    feat = pd.read_csv(FEAT_PATH)
    # Asegurar que existe target Purchase_Again
    assert 'Purchase_Again' in feat.columns, 'Falta la columna Purchase_Again en features.'

    # Modelo de retención
    clf, rep = train_retention_model(feat, target='Purchase_Again')
    print('Retención ->', rep)
    save_model(clf, 'retention_model.pkl')

    # Modelo de frecuencia (simulación: usar Ticket_Price como proxy de visitas esperadas solo para demo)
    y_reg = (feat['Ticket_Price'] / feat['Ticket_Price'].median()).clip(lower=0.2, upper=4.0)
    reg, reg_metrics = train_frequency_model(feat.drop(columns=['Purchase_Again']), y_reg)
    print('Frecuencia ->', reg_metrics)
    save_model(reg, 'frequency_model.pkl')
except Exception as e:
    print('AVISO:', e)

In [ ]:
# Explicabilidad con SHAP para el modelo de retención (Random Forest)
import shap
shap.initjs()

try:
    X = feat.drop(columns=['Purchase_Again'])
    # Muestreo para rendimiento
    X_sample = X.sample(n=min(500, len(X)), random_state=42)

    # Usar el pipeline directamente
    explainer = shap.Explainer(clf, X_sample)
    shap_values = explainer(X_sample)

    # Importancia global (bar)
    shap.plots.bar(shap_values, max_display=15)

    # Distribución detallada (beeswarm)
    shap.plots.beeswarm(shap_values, max_display=15)
except Exception as e:
    print('AVISO SHAP:', e)